In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv("/Users/farhat/Documents/Project/ProcessedData/fullData.csv")
data.head()

,Patient_Id,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,sig_30,sig_31,sig_32,sig_33,sig_34,sig_35,sig_36,sig_37,sig_38,SepsisLabel
0,p000001,97.0,95.0,36.11,98.0,75.33,63.995,19.0,NaN,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,p000001,97.0,95.0,36.11,98.0,75.33,63.995,19.0,NaN,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,p000001,89.0,99.0,36.11,122.0,86.00,68.000,22.0,NaN,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,p000001,90.0,95.0,36.11,122.0,86.00,68.000,30.0,NaN,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,p000001,103.0,88.5,36.11,122.0,91.33,75.995,24.5,NaN,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
id_unique = data['Patient_Id'].unique()
new_data = []
new_labels = []
for i in range(data.shape[0]-1):
    new_data.append(data[data['Patient_Id']==id_unique[i]].iloc[:,:-1].reset_index(drop=True))
    new_labels.append(data[data['Patient_Id']==id_unique[i]].iloc[:,-1].reset_index(drop=True))

IndexError: index 40336 is out of bounds for axis 0 with size 40336

In [39]:
new_data[40335].shape

(35, 83)

In [40]:
"""
Sepsis hourly sequence‑to‑sequence prediction with LSTM + Attention (PyTorch)
-----------------------------------------------------------------------------
This script trains a temporal attention model that predicts a binary sepsis
label for EACH HOUR of a patient’s ICU stay (seq2seq).

It assumes your data are in *long format* like:

    Patient_Id | ICULOS | <feature_1> ... <feature_K> | SepsisLabel

Key features:
- Patient‑wise train/val/test split (no leakage)
- Per‑patient sequences with padding and masks
- LSTM (bidirectional) + simple temporal attention
- Masked loss so padded hours don’t affect training
- Metrics: AUROC, AUPRC, F1 at the timestep level (masked)

How to use:
1) Put a combined CSV at DATA_CSV (one row per patient‑hour), OR
   set USE_EXISTING_FULLDATA=True to use an existing `fulldata` DataFrame
   already in the notebook environment (same columns).
2) Adjust FEATURE_COLS if you want to include/exclude vars.
3) Run the script.

Notes:
- For very imbalanced labels, consider enabling FOCAL_LOSS or POS_WEIGHT.
- If your label column is named differently, set LABEL_COL accordingly.
"""

import os
import math
import random
import warnings
from typing import List, Tuple

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========================
# Configuration
# ========================
DATA_CSV = "/Users/farhat/Documents/Project/ProcessedData/fullData.csv"   # Path to long-format CSV (ignored if USE_EXISTING_FULLDATA=True)
ID_COL = "Patient_Id"
TIME_COL = "ICULOS"         # integer hours
LABEL_COL = "SepsisLabel"   # 0/1 per hour
USE_EXISTING_FULLDATA = False  # set True if a DataFrame `fulldata` exists in memory

# Model & training
HIDDEN_DIM = 128
NUM_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.2
LR = 1e-3
BATCH_SIZE = 32
EPOCHS = 15
PATIENCE = 3                 # early stopping patience

# Loss handling for imbalance
USE_FOCAL_LOSS = False
FOCAL_GAMMA = 2.0
POS_WEIGHT = None            # e.g., torch.tensor([5.0]) to up‑weight positives; set after computing prevalence

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ========================
# Data loading / preprocessing
# ========================

def load_fulldata() -> pd.DataFrame:
    if USE_EXISTING_FULLDATA and 'fulldata' in globals():
        df = globals()['fulldata'].copy()
    else:
        if not os.path.exists(DATA_CSV):
            raise FileNotFoundError(f"DATA_CSV not found: {DATA_CSV}\nProvide a combined long‑format CSV or set USE_EXISTING_FULLDATA=True.")
        df = pd.read_csv(DATA_CSV)
    # basic checks
    for col in [ID_COL, TIME_COL, LABEL_COL]:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in data.")
    # sort
    df = df.sort_values([ID_COL, TIME_COL]).reset_index(drop=True)
    return df


def get_feature_cols(df: pd.DataFrame) -> List[str]:
    # all non‑id/time/label numeric columns are features
    excl = {ID_COL, TIME_COL, LABEL_COL}
    feature_cols = [c for c in df.columns if c not in excl]
    # keep only numeric
    feature_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df[c])]
    if not feature_cols:
        raise ValueError("No numeric feature columns found.")
    return feature_cols


def patient_split(df: pd.DataFrame, test_size=0.2, val_size=0.1) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    pids = df[ID_COL].unique()
    train_pids, test_pids = train_test_split(pids, test_size=test_size, random_state=SEED, shuffle=True)
    # split train into train/val
    train_pids, val_pids = train_test_split(train_pids, test_size=val_size, random_state=SEED, shuffle=True)
    return train_pids, val_pids, test_pids


def per_patient_impute(group: pd.DataFrame, feature_cols: List[str]) -> pd.DataFrame:
    # forward‑fill then back‑fill within a patient; then fill remaining with column medians
    group = group.copy()
    group[feature_cols] = group[feature_cols].ffill().bfill()
    return group


def compute_feature_stats(df: pd.DataFrame, feature_cols: List[str]) -> Tuple[pd.Series, pd.Series]:
    means = df[feature_cols].mean()
    stds = df[feature_cols].std(ddof=0).replace(0, 1.0)
    return means, stds


def normalize_df(df: pd.DataFrame, feature_cols: List[str], means: pd.Series, stds: pd.Series) -> pd.DataFrame:
    df = df.copy()
    df[feature_cols] = (df[feature_cols] - means) / stds
    df[feature_cols] = df[feature_cols].fillna(0.0)
    return df


def build_sequences(df: pd.DataFrame, feature_cols: List[str]) -> Tuple[List[np.ndarray], List[np.ndarray]]:
    X_list, y_list = [], []
    for pid, g in df.groupby(ID_COL, sort=False):
        g = g.sort_values(TIME_COL)
        X = g[feature_cols].to_numpy(dtype=np.float32)
        y = g[LABEL_COL].to_numpy(dtype=np.float32)
        X_list.append(X)
        y_list.append(y)
    return X_list, y_list


def pad_and_mask(X_list: List[np.ndarray], y_list: List[np.ndarray]) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    max_len = max(len(x) for x in X_list)
    feat_dim = X_list[0].shape[1]
    N = len(X_list)
    X_pad = np.zeros((N, max_len, feat_dim), dtype=np.float32)
    y_pad = np.zeros((N, max_len), dtype=np.float32)
    mask = np.zeros((N, max_len), dtype=np.float32)  # 1 for valid timesteps
    for i, (x, y) in enumerate(zip(X_list, y_list)):
        L = len(x)
        X_pad[i, :L, :] = x
        y_pad[i, :L] = y
        mask[i, :L] = 1.0
    return torch.tensor(X_pad), torch.tensor(y_pad), torch.tensor(mask)


# ========================
# Dataset / DataLoader
# ========================
class SeqDataset(Dataset):
    def __init__(self, X: torch.Tensor, y: torch.Tensor, mask: torch.Tensor):
        self.X = X
        self.y = y
        self.mask = mask
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.mask[idx]


# ========================
# Model: LSTM + Temporal Attention (per‑timestep preds)
# ========================
class LSTMAttnSeq2Seq(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, num_layers: int, bidirectional: bool = True, dropout: float = 0.0):
        super().__init__()
        self.bidirectional = bidirectional
        self.hidden_dim = hidden_dim
        self.num_dirs = 2 if bidirectional else 1
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True,
                            dropout=dropout if num_layers > 1 else 0.0, bidirectional=bidirectional)
        # simple per‑timestep attention over hidden features
        self.attn = nn.Linear(self.num_dirs * hidden_dim, 1)
        self.proj = nn.Linear(self.num_dirs * hidden_dim, 1)

    def forward(self, x):
        # x: (B, T, F)
        h, _ = self.lstm(x)  # (B, T, H*dirs)
        w = torch.softmax(self.attn(h), dim=1)  # (B, T, 1) attention over time
        context = h * w  # (B, T, H*dirs) — keeps per‑timestep but reweights
        logits = self.proj(context).squeeze(-1)  # (B, T)
        return logits, w  # return raw logits (use BCEWithLogitsLoss)


# ========================
# Loss (masked)
# ========================
class MaskedBCEWithLogitsLoss(nn.Module):
    def __init__(self, pos_weight=None):
        super().__init__()
        self.loss = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction='none')
    def forward(self, logits, targets, mask):
        # logits/targets: (B, T); mask: (B, T)
        loss = self.loss(logits, targets)
        loss = loss * mask
        denom = mask.sum().clamp_min(1.0)
        return loss.sum() / denom


class MaskedFocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        self.bce = nn.BCEWithLogitsLoss(reduction='none')
    def forward(self, logits, targets, mask):
        bce = self.bce(logits, targets)
        p = torch.sigmoid(logits)
        pt = torch.where(targets == 1, p, 1 - p)
        focal = (1 - pt).pow(self.gamma) * bce
        focal = focal * mask
        denom = mask.sum().clamp_min(1.0)
        return focal.sum() / denom


# ========================
# Metrics (masked)
# ========================
@torch.no_grad()
def masked_metrics(logits, targets, mask) -> dict:
    # flatten valid positions only
    probs = torch.sigmoid(logits).detach().cpu().numpy().ravel()
    y_true = targets.detach().cpu().numpy().ravel()
    m = mask.detach().cpu().numpy().ravel().astype(bool)

    probs = probs[m]
    y_true = y_true[m]

    out = {}
    if y_true.size == 0 or len(np.unique(y_true)) < 2:
        out['auroc'] = np.nan
    else:
        out['auroc'] = roc_auc_score(y_true, probs)
    try:
        out['auprc'] = average_precision_score(y_true, probs)
    except Exception:
        out['auprc'] = np.nan

    preds = (probs >= 0.5).astype(int)
    try:
        out['f1'] = f1_score(y_true, preds)
    except Exception:
        out['f1'] = np.nan
    return out


# ========================
# Training / Evaluation loops
# ========================

def run_epoch(model, loader, criterion, optimizer=None):
    is_train = optimizer is not None
    model.train(is_train)
    total_loss = 0.0
    n_tokens = 0.0
    all_logits, all_targets, all_masks = [], [], []

    for X, y, mask in loader:
        X = X.to(device)
        y = y.to(device)
        mask = mask.to(device)

        logits, _ = model(X)
        loss = criterion(logits, y, mask)

        if is_train:
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

        total_loss += loss.item() * mask.sum().item()
        n_tokens += mask.sum().item()
        all_logits.append(logits.detach())
        all_targets.append(y.detach())
        all_masks.append(mask.detach())

    epoch_loss = total_loss / max(n_tokens, 1.0)

    logits_cat = torch.cat(all_logits, dim=0)
    targets_cat = torch.cat(all_targets, dim=0)
    masks_cat = torch.cat(all_masks, dim=0)
    mets = masked_metrics(logits_cat, targets_cat, masks_cat)
    mets['loss'] = epoch_loss
    return mets


def main():
    print("Loading data…")
    df = load_fulldata()
    feature_cols = get_feature_cols(df)

    # Split by patients
    tr_pids, va_pids, te_pids = patient_split(df)

    # Per‑patient imputation (ffill/bfill)
    medians = df[feature_cols].median()
    df = df.groupby(ID_COL, group_keys=False).apply(lambda g: per_patient_impute(g, feature_cols))
    df[feature_cols] = df[feature_cols].fillna(medians)

    # Fit normalization on TRAIN ONLY
    train_df = df[df[ID_COL].isin(tr_pids)]
    means, stds = compute_feature_stats(train_df, feature_cols)

    # Normalize all splits with train stats
    df_norm = normalize_df(df, feature_cols, means, stds)

    # Build sequences for each split
    X_tr_list, y_tr_list = build_sequences(df_norm[df_norm[ID_COL].isin(tr_pids)], feature_cols)
    X_va_list, y_va_list = build_sequences(df_norm[df_norm[ID_COL].isin(va_pids)], feature_cols)
    X_te_list, y_te_list = build_sequences(df_norm[df_norm[ID_COL].isin(te_pids)], feature_cols)

    # Optional: compute pos_weight from TRAIN labels
    global POS_WEIGHT
    if POS_WEIGHT is None:
        pos = sum(y.sum() for y in y_tr_list)
        neg = sum(len(y) - y.sum() for y in y_tr_list)
        if pos > 0:
            POS_WEIGHT = torch.tensor([max(1.0, neg / max(pos, 1.0))], device=device)
            print(f"Using POS_WEIGHT={POS_WEIGHT.item():.3f}")
        else:
            POS_WEIGHT = None

    # Pad and mask
    X_tr, y_tr, m_tr = pad_and_mask(X_tr_list, y_tr_list)
    X_va, y_va, m_va = pad_and_mask(X_va_list, y_va_list)
    X_te, y_te, m_te = pad_and_mask(X_te_list, y_te_list)

    # Dataloaders
    train_loader = DataLoader(SeqDataset(X_tr, y_tr, m_tr), batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(SeqDataset(X_va, y_va, m_va), batch_size=BATCH_SIZE, shuffle=False)
    test_loader  = DataLoader(SeqDataset(X_te, y_te, m_te), batch_size=BATCH_SIZE, shuffle=False)

    # Model
    input_dim = X_tr.shape[2]
    model = LSTMAttnSeq2Seq(input_dim, HIDDEN_DIM, NUM_LAYERS, BIDIRECTIONAL, DROPOUT).to(device)

    # Loss
    if USE_FOCAL_LOSS:
        criterion = MaskedFocalLoss(gamma=FOCAL_GAMMA)
    else:
        criterion = MaskedBCEWithLogitsLoss(pos_weight=POS_WEIGHT)

    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    best_val = math.inf
    patience = PATIENCE

    print("Training…")
    for epoch in range(1, EPOCHS + 1):
        train_mets = run_epoch(model, train_loader, criterion, optimizer)
        val_mets = run_epoch(model, val_loader, criterion, optimizer=None)

        print(f"Epoch {epoch:02d} | train loss {train_mets['loss']:.4f} auroc {train_mets['auroc']:.4f} auprc {train_mets['auprc']:.4f} f1 {train_mets['f1']:.4f} | "
              f"val loss {val_mets['loss']:.4f} auroc {val_mets['auroc']:.4f} auprc {val_mets['auprc']:.4f} f1 {val_mets['f1']:.4f}")

        if val_mets['loss'] < best_val - 1e-4:
            best_val = val_mets['loss']
            patience = PATIENCE
            torch.save({'model_state': model.state_dict(), 'config': {
                'input_dim': input_dim,
                'hidden_dim': HIDDEN_DIM,
                'num_layers': NUM_LAYERS,
                'bidirectional': BIDIRECTIONAL,
                'dropout': DROPOUT
            }}, 'best_model.pt')
        else:
            patience -= 1
            if patience <= 0:
                print("Early stopping.")
                break

    print("Loading best model and evaluating on TEST…")
    ckpt = torch.load('best_model.pt', map_location=device)
    model.load_state_dict(ckpt['model_state'])

    test_mets = run_epoch(model, test_loader, criterion, optimizer=None)
    print({k: (None if isinstance(v, float) and (np.isnan(v) or np.isinf(v)) else v) for k, v in test_mets.items()})

    # Optionally, save per‑timestep probabilities for test set
    all_probs = []
    all_true = []
    all_masks = []
    model.eval()
    with torch.no_grad():
        for X, y, mask in test_loader:
            X = X.to(device)
            logits, _ = model(X)
            probs = torch.sigmoid(logits).cpu().numpy()
            all_probs.append(probs)
            all_true.append(y.numpy())
            all_masks.append(mask.numpy())
    np.save('test_probs.npy', np.concatenate(all_probs, axis=0))
    np.save('test_true.npy',  np.concatenate(all_true, axis=0))
    np.save('test_mask.npy',  np.concatenate(all_masks, axis=0))
    print("Saved test_probs.npy, test_true.npy, test_mask.npy")


if __name__ == "__main__":
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=UserWarning)
        main()


Loading data…


/var/folders/zc/cx2984q17xs9k62vy4ms148w0000gn/T/ipykernel_14293/68976783.py:313: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(ID_COL, group_keys=False).apply(lambda g: per_patient_impute(g, feature_cols))


Using POS_WEIGHT=53.581
Training…
Epoch 01 | train loss 1.3129 auroc 0.6981 auprc 0.0380 f1 0.0738 | val loss 1.2645 auroc 0.7452 auprc 0.0533 f1 0.0929
Epoch 02 | train loss 1.0806 auroc 0.8390 auprc 0.0800 f1 0.1056 | val loss 0.8634 auroc 0.9224 auprc 0.2280 f1 0.1491
Epoch 03 | train loss 0.8294 auroc 0.9142 auprc 0.1373 f1 0.1681 | val loss 0.7432 auroc 0.9294 auprc 0.1925 f1 0.1527
Epoch 04 | train loss 0.7247 auroc 0.9341 auprc 0.1796 f1 0.2005 | val loss 0.6251 auroc 0.9501 auprc 0.2742 f1 0.2449
Epoch 05 | train loss 0.5902 auroc 0.9532 auprc 0.2461 f1 0.2315 | val loss 0.5430 auroc 0.9624 auprc 0.3624 f1 0.2556
Epoch 06 | train loss 0.5407 auroc 0.9599 auprc 0.2918 f1 0.2589 | val loss 0.4993 auroc 0.9641 auprc 0.3470 f1 0.2447
Epoch 07 | train loss 0.4743 auroc 0.9684 auprc 0.3552 f1 0.2860 | val loss 0.6452 auroc 0.9569 auprc 0.4395 f1 0.2892
Epoch 08 | train loss 0.8742 auroc 0.8945 auprc 0.1836 f1 0.1618 | val loss 1.0940 auroc 0.8993 auprc 0.2634 f1 0.2008
Epoch 09 | tra